# Training YOLO on Fire Detection for Far-Field Scenario

## Overview
In this notebook, we train a **YOLOv8** model on a custom dataset designed for detecting **fire scenarios** in **far-field images**. This dataset represents one of the specific scenarios for our **Mixture of Experts (MoE)** model, where each expert specializes in a different scenario (e.g., fire detection in outdoor, indoor, satellite, or far-field environments).

### Key Steps:
- **Dataset**: The model is trained using a **far-field fire detection dataset**, which consists of images with fire-related features captured in distant or large-scale scenarios.
- **YOLOv8 Training**: The YOLOv8 model is fine-tuned on this dataset, learning to identify fire-related objects.
- **Scenario Expert**: This trained model acts as an expert specifically for detecting fire in far-field images and is part of a broader MoE-based approach for multi-scenario detection.



##  Data Loading & Preprocessing


In [1]:
from datasets import load_dataset

# Load the Pyro-SDIS dataset from Hugging Face
ds = load_dataset("pyronear/pyro-sdis")


C:\Users\pc\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import random

# Set a seed for reproducibility
random.seed(42)

# Select 1000 random samples from train
train_subset = ds['train'].shuffle(seed=42).select(range(1000))

# Select 100 random samples from validation
val_subset = ds['val'].shuffle(seed=42).select(range(100))


In [4]:
import os
from PIL import Image

# Base directory
base_dir = "pyro_subset_yolo"
os.makedirs(base_dir, exist_ok=True)

# Helper function to save images and labels
def save_subset_to_yolo_format(subset, split):
    img_dir = os.path.join(base_dir, "images", split)
    label_dir = os.path.join(base_dir, "labels", split)
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for item in subset:
        # Save image
        image = item["image"]
        img_path = os.path.join(img_dir, item["image_name"])
        image.save(img_path)

        # Save annotation (single line YOLO format)
        label_path = os.path.join(label_dir, item["image_name"].replace(".jpg", ".txt"))
        with open(label_path, "w") as f:
            f.write(item["annotations"])

    print(f"Saved {split} set to {img_dir} and {label_dir}")

# Save both subsets
save_subset_to_yolo_format(train_subset, "train")
save_subset_to_yolo_format(val_subset, "val")


Saved train set to pyro_subset_yolo\images\train and pyro_subset_yolo\labels\train
Saved val set to pyro_subset_yolo\images\val and pyro_subset_yolo\labels\val


In [6]:
import os

# Path to your labels directory
labels_dir = "C:/Users/pc/Desktop/DL/Project/FarField/pyro_subset_yolo/labels/train"

# Iterate through label files
for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)

    if os.path.isfile(label_path) and label_file.endswith(".txt"):
        with open(label_path, 'r') as file:
            lines = file.readlines()

        # Replace class 1 with class 0
        lines = [line.replace('1 ', '0 ') for line in lines]

        # Save the modified file
        with open(label_path, 'w') as file:
            file.writelines(lines)

        print(f"Updated {label_file} to class 0")


Updated force-06_cabanelle-125_2024-01-04T08-34-18.txt to class 0
Updated force-06_cabanelle-125_2024-01-08T13-12-25.txt to class 0
Updated force-06_cabanelle-125_2024-02-21T07-53-21.txt to class 0
Updated force-06_cabanelle-125_2024-02-21T07-54-51.txt to class 0
Updated force-06_cabanelle-125_2024-02-21T08-29-22.txt to class 0
Updated force-06_cabanelle-125_2024-02-21T08-30-23.txt to class 0
Updated force-06_cabanelle-125_2024-02-21T08-36-53.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T13-31-25.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T13-48-25.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T14-11-26.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T14-18-56.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T14-45-58.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T14-46-27.txt to class 0
Updated force-06_cabanelle-125_2024-02-27T14-48-57.txt to class 0
Updated force-06_cabanelle-125_2024-02-28T10-35-52.txt to class 0
Updated fo

In [7]:
import os

# Path to your labels directory
labels_dir = "C:/Users/pc/Desktop/DL/Project/FarField/pyro_subset_yolo/labels/val"

# Iterate through label files
for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)

    if os.path.isfile(label_path) and label_file.endswith(".txt"):
        with open(label_path, 'r') as file:
            lines = file.readlines()

        # Replace class 1 with class 0
        lines = [line.replace('1 ', '0 ') for line in lines]

        # Save the modified file
        with open(label_path, 'w') as file:
            file.writelines(lines)

        print(f"Updated {label_file} to class 0")


Updated force-06_cabanelle-125_2024-02-24T09-07-28.txt to class 0
Updated force-06_cabanelle-244_2024-09-19T14-03-44.txt to class 0
Updated force-06_cabanelle-327_2024-01-01T16-49-40.txt to class 0
Updated force-06_cabanelle-327_2024-02-23T08-55-55.txt to class 0
Updated force-06_cabanelle-327_2024-04-02T18-49-16.txt to class 0
Updated force-06_cabanelle-327_2024-04-02T18-59-14.txt to class 0
Updated force-06_cabanelle-327_2024-04-04T19-29-52.txt to class 0
Updated force-06_courmettes-160_2024-01-08T12-56-37.txt to class 0
Updated force-06_courmettes-160_2024-02-13T12-56-15.txt to class 0
Updated force-06_courmettes-160_2024-02-13T14-10-16.txt to class 0
Updated force-06_courmettes-160_2024-04-03T08-36-45.txt to class 0
Updated force-06_courmettes-160_2024-04-03T09-51-07.txt to class 0
Updated force-06_courmettes-160_2024-04-03T09-56-17.txt to class 0
Updated force-06_courmettes-160_2024-04-03T10-22-47.txt to class 0
Updated force-06_courmettes-160_2024-04-03T11-33-33.txt to class 0
Up

##  Training YOLOv8 on Far-Field Fire Detection

We utilize the **YOLOv8** object detection model to train on a custom dataset defined in `farfield.yaml`. The configuration for training is as follows:

- 🧠 **Model**: `yolov8n.pt` (Nano variant — optimized for speed and prototyping)
- 📁 **Dataset**: Defined in `farfield.yaml`
- 🖼️ **Image Size**: 640×640
- 🔁 **Epochs**: 100
- 📦 **Batch Size**: 16
- 💻 **Device**: GPU (device 0)

This setup helps the model learn to detect **fire in far-field scenarios**, which often includes distant flames or smoke in wide-view environments.


In [8]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

GPU available: True
Current device: NVIDIA GeForce RTX 3070


In [9]:
from ultralytics import YOLO

# Load YOLOv8 model (you can try 'yolov8n.pt', 'yolov8s.pt', etc.)
model = YOLO('yolov8n.pt')  # 'n' is the nano model — fastest for prototyping

# Train the model
model.train(
    data="farfield.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    device = 0

)


Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=farfield.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=Non

train: Scanning C:\Users\pc\Desktop\DL\Project\FarField\pyro_subset_yolo\labels\train... 1000 images, 167 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:01<00:00, 605.86it/s]


train: New cache created: C:\Users\pc\Desktop\DL\Project\FarField\pyro_subset_yolo\labels\train.cache


val: Scanning C:\Users\pc\Desktop\DL\Project\FarField\pyro_subset_yolo\labels\val... 100 images, 17 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 391.28it/s]

val: New cache created: C:\Users\pc\Desktop\DL\Project\FarField\pyro_subset_yolo\labels\val.cache


...

                   all        100         98      0.679      0.643      0.624      0.336



100 epochs completed in 0.209 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  7.04it/s]


                   all        100         98      0.626      0.684      0.666      0.368
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002503C4B7A50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [11]:
import random

# Set a seed for reproducibility
random.seed(42)

# Select 50 random samples from the validation set
test_subset = ds['val'].shuffle(seed=42).select(range(50))

# If you want to save the test set, for instance as image paths and annotations
test_images = test_subset['image']
test_annotations = test_subset['annotations']

# You can print or inspect the selected samples
print(f"Test set size: {len(test_images)}")


Test set size: 50


In [16]:
import os
import shutil
from PIL import Image

# Define the path where the test images will be saved
test_dir = 'test/images'

# Create the directory if it doesn't exist
os.makedirs(test_dir, exist_ok=True)

# Save the test images to the test/images directory
for idx, img in enumerate(test_images):
    img.save(os.path.join(test_dir, f"test_image_{idx+1}.jpg"))  # Save with a new name

print(f"Test images have been saved to {test_dir}")

Test images have been saved to test/images


##  Inference on Far-Field Test Images using Trained YOLOv8 Model

We perform inference using the **best trained YOLOv8 model** to evaluate its performance on the **test set** of far-field fire scenarios.

- 🎯 **Model Weights**: `best.pt` from the training run
- 💾 **Output**: Predictions will be saved automatically by YOLO

This step visually validates how well the model generalizes to unseen far-field fire detection cases.


In [17]:
from ultralytics import YOLO

# Load your trained weights
model = YOLO("runs/detect/train/weights/best.pt")


In [18]:
results = model.predict(source="C:/Users/pc/Desktop/DL/Project/FarField/test/images", save=True)



image 1/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_1.jpg: 384x640 1 fire, 47.0ms
image 2/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_10.jpg: 384x640 1 fire, 7.5ms
image 3/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_11.jpg: 384x640 1 fire, 6.5ms
image 4/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_12.jpg: 384x640 1 fire, 5.4ms
image 5/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_13.jpg: 384x640 2 fires, 5.0ms
image 6/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_14.jpg: 384x640 1 fire, 6.7ms
image 7/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_15.jpg: 384x640 1 fire, 5.5ms
image 8/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_16.jpg: 384x640 (no detections), 6.7ms
image 9/50 C:\Users\pc\Desktop\DL\Project\FarField\test\images\test_image_17.jpg: 384x640 (no detections), 8.6ms
image 10/50 C:\Users\pc\Desktop\DL\Project\F

In [20]:
val_metrics = model.val(data="farfield.yaml", imgsz=640)

# Print or log test metrics
print("\n🔍 Test Data Evaluation Metrics:")
print(f"Precision:      {val_metrics.box.p.mean():.4f}")
print(f"Recall:         {val_metrics.box.r.mean():.4f}")
print(f"mAP@0.5:        {val_metrics.box.map50.mean():.4f}")
print(f"mAP@0.5:0.95:   {val_metrics.box.map.mean():.4f}")

Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)


val: Scanning C:\Users\pc\Desktop\DL\Project\FarField\pyro_subset_yolo\labels\val.cache... 100 images, 17 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.09it/s]


                   all        100         98      0.633      0.685      0.667       0.37
Speed: 1.5ms preprocess, 4.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\val2

🔍 Test Data Evaluation Metrics:
Precision:      0.6327
Recall:         0.6855
mAP@0.5:        0.6674
mAP@0.5:0.95:   0.3701
